<a href="https://colab.research.google.com/github/Simon-creator-jara/Inteligencia_artificial/blob/main/Copia_de_Juego_de_IA_MIN_MAX_ALFA_BETA_ultima_recipiente.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pydot
from IPython.display import Image, display
import queue
import numpy as np
import random

Podemos reusar la clase nodo y árbol desarrollada para los algoritmos de búsqueda a ciega y búsqueda heurística. Adicionando el método minimax y alfa-beta a la última

In [ ]:
class Node ():
  def __init__(self, state,value,operators,operator=None, parent=None,objective=None):
    self.state= state
    self.value = value
    self.children = []
    self.parent=parent
    self.operator=operator
    self.objective=objective
    self.level=0
    self.operators=operators
    self.v=0

    
  def add_child(self, value, state, operator):
    node=type(self)(value=value, state=state, operator=operator,parent=self,operators=self.operators)
    node.level=node.parent.level+1
    self.children.append(node)
    return node
  
  def add_node_child(self, node):
    node.level=node.parent.level+1
    self.children.append(node)    
    return node

  #Devuelve todos los estados según los operadores aplicados
  def getchildrens(self):
    return [
        self.getState(i) 
          if not self.repeatStatePath(self.getState(i)) 
            else None for i, op in enumerate(self.operators)]
    
  def getState(self, index):
    pass
  
  def __eq__(self, other):
    return self.state == other.state
 
  def __lt__(self, other):
    return self.f() < other.f()
   
  
  def repeatStatePath(self, state):
      n=self
      while n is not None and n.state!=state:
          n=n.parent
      return n is not None
    
  def pathObjective(self):
      n=self
      result=[]
      while n is not None:
          result.append(n)
          n=n.parent
      return result
  
  def heuristic(self):
    return 0
  
  def cost(self):
    return 1
  
  def f(self): 
    return self.cost()+self.heuristic()

  ### Crear método para criterio objetivo
  ### Por defecto vamos a poner que sea igual al estado objetivo, para cada caso se puede sobreescribir la función
  def isObjective(self):
    return (self.state==self.objetive.state)


In [ ]:
class Tree ():
  def __init__(self, root ,operators):
    self.root=root
    self.operators=operators

  def printPath(self,n):
    stack=n.pathObjective()
    path=stack.copy()
    while len(stack)!=0:
        node=stack.pop()
        if node.operator is not None:
            print(f'operador:  {self.operators[node.operator]} \t estado: {node.state}')
        else:
            print(f' {node.state}')
    return path

  def Path(self,n):
    stack=n.pathObjective()
    path=stack.copy()
    while len(stack)!=0:
        node=stack.pop()
        if node.operator is not None:
            r=node.state

    return r


  def reinitRoot(self):
    self.root.operator=None
    self.root.parent=None
    self.root.objective=None
    self.root.children = []
    self.root.level=0

  ## Primero a lo ancho
  def breadthFirst(self,endState):
    self.reinitRoot()
    pq=queue.Queue()
    pq.put(self.root)
    while not pq.empty():
      node=pq.get()
      children=node.getchildrens()
      for i,child in enumerate(children):
        if child is not None:
          newChild=node.add_child(value=node.value+'-'+str(i), state=child, operator=i)
          pq.put(newChild)
          if endState==child:
            return newChild

  ## Primero en profundidad
  def dephFirst(self, endState):
    self.reinitRoot()
    pq=[]
    pq.append(self.root)    
    while len(pq)>0:
      node=pq.pop()
      if (node.parent is not None):
        node.parent.add_node_child(node)
      children=node.getchildrens()
      temp=[]
      for i,child in enumerate(children):
        if child is not None:
          newChild=type(self.root)(value=node.value+'-'+str(i),state=child,operator=i,parent=node, operators=node.operators)
          temp.append(newChild)
          if endState==child:
            node.add_node_child(newChild)
            return newChild
      #Adicionar los hijos en forma inversa para que salga primero el primero que se adicionó
      temp.reverse()
      for e in temp:
        pq.append(e)
  
  ## Costo uniforme
  def costUniform(self,endState):
    self.reinitRoot()
    pq=queue.PriorityQueue()
    pq.put((self.root.cost(),self.root))
    while not pq.empty():
      node=pq.get()[1]
      children=node.getchildrens()
      for i,child in enumerate(children):
        if child is not None:
          newChild=node.add_child(value=node.value+'-'+str(i), 
                                  state=child, operator=i)
          pq.put((newChild.cost(),newChild))
          if endState==child:
            return newChild

  ## Primero el mejor 
  def bestFirst(self,endState):
    self.reinitRoot()
    pq=queue.PriorityQueue()
    pq.put((self.root.heuristic(),self.root))
    while not pq.empty():
      node=pq.get()[1]
      children=node.getchildrens()
      for i,child in enumerate(children):
        if child is not None:
          newChild=node.add_child(value=node.value+'-'+str(i), 
                                  state=child, operator=i)
          pq.put((newChild.heuristic(),newChild))
          if endState==child:
            return newChild

  ## A*
  def Aasterisk(self,endState):
    self.reinitRoot()
    pq=queue.PriorityQueue()
    pq.put((self.root.f(),self.root))
    while not pq.empty():
      node=pq.get()[1]
      children=node.getchildrens()
      for i,child in enumerate(children):
        if child is not None:
          newChild=node.add_child(value=node.value+'-'+str(i), 
                                  state=child, operator=i)
          pq.put((newChild.f(),newChild))
          if endState==child:
            return newChild

    ## Generar los hijos del nodo 
    
  def miniMax(self, depth,alpha,beta):
    self.root.v=self.miniMaxR(self.root, depth,alpha,beta,True)
    print(f"La raíz es:{self.root.state}")
    print(f"el valor es:{self.root.v}")
    ## Comparar los hijos de root
    values=[c.v for c in self.root.children]
    print(f"Los valores son:{values}")
    maxvalue=max(values)
    print(f"Maximo valor es:{maxvalue}")
    index=values.index(maxvalue)
    return self.root.children[index]

  def miniMaxR(self, node, depth, alpha,beta,maxPlayer):
    if depth==0 or node.isObjective():
      node.v=node.heuristic()
      return node.heuristic()
    ## Generar los hijos del nodo
    children=node.getchildrens()
    
    ## Según el jugador que sea en el árbol
    if maxPlayer:
      value=float('-inf')
  
      for i,child in enumerate(children):
        if child is not None:
          newChild=type(self.root)(value=node.value+'-'+str(i),state=child,operator=i,parent=node, operators=node.operators,player=False)
          newChild=node.add_node_child(newChild)
          value=max(value,self.miniMaxR(newChild,depth-1,alpha,beta,False))
          alpha=max(alpha,self.miniMaxR(newChild,depth-1,alpha,beta,False))
          if beta <=alpha:
            break
      #node.v=value
      #return value
    else:
      value=float('inf')

      for i,child in enumerate(children):
        if child is not None:
          newChild=type(self.root)(value=node.value+'-'+str(i),state=child,operator=i,parent=node, operators=node.operators,player=True)
          newChild=node.add_node_child(newChild)
          value=min(value,self.miniMaxR(newChild,depth-1,alpha,beta,True))
          beta=min(beta,self.miniMaxR(newChild,depth-1,alpha,beta,True))
          if beta <=alpha:
            break
    node.v=value
    return value

  ## Método para dibujar el árbol        
  def draw(self,path):
    graph = pydot.Dot(graph_type='graph')
    nodeGraph=pydot.Node(str(self.root.state)+"-"+str(0),
                          label=str(self.root.state),shape ="circle", 
                          style="filled", fillcolor="red")
    graph.add_node(nodeGraph)
    path.pop()
    return self.drawTreeRec(self.root,nodeGraph,graph,0,path.pop(),path)
  
  ## Método recursivo para dibujar el árbol  
  def drawTreeRec(self,r,rootGraph,graph,i,topPath,path):
    if r is not None:
      children=r.children
      for j,child in enumerate(children):
        i=i+1
        color="white"
        if topPath.value==child.value:
          if len(path)>0:topPath=path.pop()
          color='red'
        c=pydot.Node(child.value,label=str(child.state)+r"\n"+r"\n"+"f="+str(child.heuristic())+r"\n"+str(child.v), 
                      shape ="circle", style="filled", 
                      fillcolor=color)
        graph.add_node(c)
        graph.add_edge(pydot.Edge(rootGraph, c, 
                                  label=str(child.operator)+'('+str(child.cost())+')'))
        graph=self.drawTreeRec(child,c,graph,i,topPath,path)  # recursive call
      return graph
    else:
      return graph  

In [ ]:
class NodeTicTacToe(Node):
  ## Vamos a añadir el jugador, pues en dependencia del jugador se hace una cosa u otra.

  def __init__(self, player=True,**kwargs):
    super(NodeTicTacToe, self).__init__(**kwargs)
    self.player=player
    if player:
      self.v=float('-inf')
    else:
      self.v=float('inf')
    

  
  def getState(self, index):
    state=self.state
    nextState=None
    # playerOne=True
    # playing=True
    lastrecipient=-1
    # messageCode=0
    giveawayPile=-1
    
    chosenBin=-1
    # print(f"estado actual:{state}")
    # print(self.player)
    # print(state[index])
   

    
    if self.player==False and int(index)==5:
      chosenBin = 5
    elif self.player==False and int(index)==4:
      chosenBin = 4
    elif self.player==False and int(index)==3:
      chosenBin = 3
    elif self.player==False and int(index)==2:
      chosenBin = 2  
    elif self.player==False and int(index)==1:
      chosenBin = 1  
    elif self.player==False and int(index)==0:
      chosenBin = 0 

  
    if self.player==True and int(index)==12:
      chosenBin = 12 
    elif self.player==True and int(index)==11:
      chosenBin = 11 
    elif self.player==True and int(index)==10:
      chosenBin = 10 
    elif self.player==True and int(index)==9:
      chosenBin = 9 
    elif self.player==True and int(index)==8:
      chosenBin = 8
    elif self.player==True and int(index)==7:
        chosenBin = 7 

    nextState=state.copy()
    if int(chosenBin)>=0:
      giveawayPile=state[chosenBin]
      nextState[chosenBin]=0
      # if int(giveawayPile) <=0:

      #   messageCode=-1 #empty bin was chosen 


    recipient= chosenBin +1
    while(int(giveawayPile)>0):
      if (self.player==False and int(recipient)==13):
        recipient=0
      if (self.player==True and int(recipient)==6):
        recipient=7
      nextState[recipient] = int(nextState[recipient])+1
      giveawayPile=int(giveawayPile)-1
  
      
      if int(giveawayPile)==0:
        lastrecipient=recipient
      else:
        recipient=int(recipient)+1
        if int(recipient)>13:
          recipient=0

    if (self.player==False and int(lastrecipient)==6):
      print(nextState)
      print("###############################Se repite########################################################")
      nodeInit=NodeTicTacToe(False,value="inicio",state=nextState, operators= self.operators)
      treeMinMax= Tree(nodeInit,self.operators)
      objective=treeMinMax.miniMax(1,float('-inf'),float('inf'))
      nextState=treeMinMax.Path(objective)

    elif(self.player==False and int(state[lastrecipient])==1 and int(lastrecipient)<6):
      nextState[6]=int(state[6])+ int(state[lastrecipient])+int(state[12-int(lastrecipient)])
      nextState[lastrecipient]=0
      nextState[12-int(lastrecipient)]=0
    elif (self.player==True and int(lastrecipient)==13):
      print(nextState)
      print("###############################Se repite########################################################")
      nodeInit=NodeTicTacToe(True,value="inicio",state=nextState, operators= self.operators)
      treeMinMax= Tree(nodeInit,self.operators)
      objective=treeMinMax.miniMax(1,float('-inf'),float('inf'))
      nextState=treeMinMax.Path(objective)      
    elif(self.player==True and int(state[lastrecipient])==1 and int(lastrecipient)>6):
      nextState[13]=int(state[13])+ int(state[lastrecipient])+int(state[12-int(lastrecipient)])
      nextState[lastrecipient]=0
      nextState[12-int(lastrecipient)]=0
   
    # print(chosenBin)
    
    #print(f"se movieron las fichas de la casilla:{index}")
    return nextState if state!=nextState else None




  #Costo acumulativo(valor 1 en cada nivel)
  def cost(self):
    return self.level
  
  ##Ver si el nodo es un nodo objetivo para O o para X, o hay empate
  def isObjective(self):
    sideOne=0
    sideTwo=0
    for j in range(6):
      sideOne=int(sideOne)+int(self.state[j])
      sideTwo=int(sideTwo)+int(self.state[j+7])

    if (int(sideOne)==0 or int(sideTwo)==0):

      return True
    else:
      return False 

  ## Si es nodo objetivo, si X retornamos 1, si O -1 y si no 0
  def heuristic(self):
    
    ans=0
    puntaje=0
    puntaje_max=self.state[13]
    puntaje_min=self.state[6]
    cont_max=self.state[7]+self.state[8]+self.state[9]+self.state[10]+self.state[11]+self.state[12]
    cont_min=self.state[0]+self.state[1]+self.state[2]+self.state[3]+self.state[4]+self.state[5]
    #if cont_max > cont_min and self.player==True:
    #   ans=ans+10
    # elif cont_max < cont_min and self.player==True:
    #   ans=ans+-10
    # elif cont_max > cont_min and self.player==False:
    #   ans=ans+-10
    # elif cont_max < cont_min and self.player==False:
    #   ans=ans+10
    
    # if puntaje_max > puntaje_min and self.player==True:
    #   ans=ans+10
    # elif puntaje_max < puntaje_min and self.player==True:
    #   ans=ans+-10
    # elif puntaje_max > puntaje_min and self.player==False:
    #   ans=ans+-10
    # elif puntaje_max < puntaje_min and self.player==False:
    #   ans=ans+10
    ## Elaborar resto de heurística...
    return cont_max-cont_min

In [ ]:
#'poner ficha en casilla vacía fila f, columna c', lista de posiciones para poner la ficha
initState=[4,4,4,4,4,4,0,4,4,4,4,4,4,0]

operators=[i for i,f in enumerate(initState) ]

In [ ]:
operators

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]

In [ ]:
nodeInit=NodeTicTacToe(True,value="inicio",state=initState, operators= operators)
treeMinMax= Tree(nodeInit,operators)

In [ ]:
objective=treeMinMax.miniMax(5,float('-inf'),float('inf'))

Se han truncado las últimas 5000 líneas del flujo de salida.
Los valores son:[-8, -8, -17, -19]
Maximo valor es:-8
[0, 0, 7, 6, 6, 6, 1, 1, 0, 2, 7, 7, 0, 5]
###############################Se repite########################################################
La raíz es:[0, 0, 7, 6, 6, 6, 1, 1, 0, 2, 7, 7, 0, 5]
el valor es:-8
Los valores son:[-8, -8, -17, -19]
Maximo valor es:-8
La raíz es:[0, 0, 7, 6, 6, 6, 1, 1, 0, 2, 7, 7, 1, 4]
el valor es:-7
Los valores son:[-7, -7, -16, -18, -8]
Maximo valor es:-7
[0, 0, 7, 6, 6, 6, 1, 1, 0, 2, 7, 7, 1, 4]
###############################Se repite########################################################
[0, 0, 7, 6, 6, 6, 1, 1, 0, 2, 7, 7, 0, 5]
###############################Se repite########################################################
La raíz es:[0, 0, 7, 6, 6, 6, 1, 1, 0, 2, 7, 7, 0, 5]
el valor es:-8
Los valores son:[-8, -8, -17, -19]
Maximo valor es:-8
[0, 0, 7, 6, 6, 6, 1, 1, 0, 2, 7, 7, 0, 5]
###############################Se repite#########

In [ ]:
matriz=treeMinMax.Path(objective)

In [ ]:
print(matriz)

[4, 4, 4, 4, 4, 4, 0, 0, 5, 5, 5, 5, 4, 0]


In [ ]:
#Interfaz gráfica
matriz=[4,4,4,4,4,4,0,4,4,4,4,4,4,0]
operators=[i for i,f in enumerate(matriz) ]
playerOne=True
playing=True

messageCode=0
giveawayPile=-1
lastrecipient=-1
chosenBin=-1
while(playing):
  operators=[i for i,f in enumerate(matriz) ] 
  lista_n=[]
  message=""
  
  if playerOne and messageCode==0:
    message="Turno jugador uno"
  elif not (playerOne) and messageCode==0:
    message="Turno máquina"
  elif playerOne and messageCode==-2:
    message="Opción inválida, escoge otra Jugador 1"
  elif not(playerOne) and messageCode==-2:
    message="Opción inválida, escoge otra máquina"
  elif playerOne and messageCode==-1:
    message="Casilla vacía, escoge otra Jugador 1"
  elif not(playerOne) and messageCode==-1:
    message="Casilla vacía, escoge otra máquina"    
  print("")
  print(message)
  print("")
  messageCode=0
  i=0
  for element in matriz:
    matriz[i]=int(matriz[i])
    if int(matriz[i]) <10:
      matriz[i] = " " + str(matriz[i])
    else:
      matriz[i] = str(matriz[i])
    i=i+1

  # if not (playerOne):
  #   print("        a    b    c    d    e    f")  
  print("+----+----+----+----+----+----+----+----+")
  print("|    | "+ matriz[12] +" | "+ matriz[11] +" | "+ matriz[10] +" | "+ matriz[9] +" | "+ matriz[8] +" | "+ matriz[7] +" |    |")
  print("| "+ matriz[13] +" |----+----+----+----+----+----| "+ matriz[6] +" |")
  print("|    | "+ matriz[0] +" | "+ matriz[1] +" | "+ matriz[2] +" | "+ matriz[3] +" | "+ matriz[4] +" | "+ matriz[5] +" |    |")
  print("+----+----+----+----+----+----+----+----+")

  if playerOne:
    print("        f    e    d    c    b    a")
  userInput=input("Enter 'q' to QUIT the game: ")


  if userInput.lower() == "q":
    playing=False
    chosenBin = 0
  elif playerOne and userInput.lower()=='a':
    chosenBin = 5
  elif playerOne and userInput.lower()=='b':
    chosenBin = 4
  elif playerOne and userInput.lower()=='c':
    chosenBin = 3
  elif playerOne and userInput.lower()=='d':
    chosenBin = 2  
  elif playerOne and userInput.lower()=='e':
    chosenBin = 1  
  elif playerOne and userInput.lower()=='f':
    chosenBin = 0 
  # elif not(playerOne) and userInput.lower()=='a':
  #   chosenBin = 12 
  # elif not(playerOne) and userInput.lower()=='b':
  #   chosenBin = 11 
  # elif not(playerOne) and userInput.lower()=='c':
  #   chosenBin = 10 
  # elif not(playerOne) and userInput.lower()=='d':
  #   chosenBin = 9 
  # elif not(playerOne) and userInput.lower()=='e':
  #   chosenBin = 8
  # elif not(playerOne) and userInput.lower()=='f':
  #   chosenBin = 7 
  else:
    chosenBin=-2
    messageCode=-2 #input inavalid

  for i in matriz:
    lista_n.append(int(i))

  matriz=lista_n
  if int(chosenBin)>=0:
    giveawayPile=matriz[chosenBin]
    matriz[chosenBin]=0
    if int(giveawayPile) <=0:

      messageCode=-1 #empty bin was chosen 

  recipient= chosenBin +1
  while(int(giveawayPile)>0):
    if (playerOne and int(recipient)==13):
      recipient=0
    # if (not(playerOne) and int(recipient)==6):
    #   recipient=7
    matriz[recipient] = int(matriz[recipient])+1
    giveawayPile=int(giveawayPile)-1
 
    
    if int(giveawayPile)==0:
      lastrecipient=recipient
    else:
      recipient=int(recipient)+1
      if int(recipient)>13:
        recipient=0

  if (playerOne and int(lastrecipient)==6):
    playerOne=True
  elif(playerOne and int(matriz[lastrecipient])==1 and int(lastrecipient)<6):
    matriz[6]=int(matriz[6])+ int(matriz[lastrecipient])+int(matriz[12-int(lastrecipient)])
    matriz[lastrecipient]=0
    matriz[12-int(lastrecipient)]=0
    try:
      nodeInit=NodeTicTacToe(True,value="inicio",state=matriz, operators= operators)
      treeMinMax= Tree(nodeInit,operators)
      objective=treeMinMax.miniMax(1,float('-inf'),float('inf'))
      matriz=treeMinMax.Path(objective)
      path=treeMinMax.printPath(objective)

    except:
      playing=False
  # elif (not(playerOne) and int(nodeInit.lastrecipient)==13):
  #   nodeInit=NodeTicTacToe(True,value="inicio",state=matriz, operators= operators)
  #   treeMinMax= Tree(nodeInit,operators)
  #   objective=treeMinMax.miniMax(1,float('-inf'),float('inf'))
  #   matriz=treeMinMax.Path(objective)
  #   path=treeMinMax.printPath(objective)
  #   playerOne=not(playerOne)

  # elif(not(playerOne) and int(matriz[lastrecipient])==1 and int(lastrecipient)>6):
  #   matriz[13]=int(matriz[13])+ int(matriz[lastrecipient])+int(matriz[12-int(lastrecipient)])
  #   matriz[lastrecipient]=0
  #   matriz[12-int(lastrecipient)]=0
  #   playerOne=not(playerOne)

  elif (int(messageCode)>=0):
    try:
      nodeInit=NodeTicTacToe(True,value="inicio",state=matriz, operators= operators)
      treeMinMax= Tree(nodeInit,operators)
      objective=treeMinMax.miniMax(1,float('-inf'),float('inf'))
      matriz=treeMinMax.Path(objective)
      path=treeMinMax.printPath(objective)
    except:
      playing=False

  
  # checking for the end of the game 
  sideOne=0
  sideTwo=0
  for j in range(6):
    sideOne=int(sideOne)+int(matriz[j])
    sideTwo=int(sideTwo)+int(matriz[j+7])

  if (int(sideOne)==0 or int(sideTwo)==0):
    playing=False
    matriz[6]=int(matriz[6]) + int(sideOne)
    matriz[13]=int(matriz[13]) + int(sideTwo)
    for k in range(6):
      matriz[k]=0
      matriz[k+7]=0


if int(matriz[13]) < int(matriz[6]):
  print("Jugador 1 gana!!!!!")

elif int(matriz[13]) > int(matriz[6]):
  print("Máquina gana!!!!!")
else:
  print("Empatados")

i=0
for element in matriz:
  matriz[i]=int(matriz[i])
  if int(matriz[i]) <10:
    matriz[i] = " " + str(matriz[i])
  else:
    matriz[i] = str(matriz[i])
  i=i+1

print("")
print("+----+----+----+----+----+----+----+----+")
print("|    | "+ matriz[12] +" | "+ matriz[11] +" | "+ matriz[10] +" | "+ matriz[9] +" | "+ matriz[8] +" | "+ matriz[7] +" |    |")
print("| "+ matriz[13] +" |----+----+----+----+----+----| "+ matriz[6] +" |")
print("|    | "+ matriz[0] +" | "+ matriz[1] +" | "+ matriz[2] +" | "+ matriz[3] +" | "+ matriz[4] +" | "+ matriz[5] +" |    |")
print("+----+----+----+----+----+----+----+----+")



Turno jugador uno

+----+----+----+----+----+----+----+----+
|    |  4 |  4 |  4 |  4 |  4 |  4 |    |
|  0 |----+----+----+----+----+----|  0 |
|    |  4 |  4 |  4 |  4 |  4 |  4 |    |
+----+----+----+----+----+----+----+----+
        f    e    d    c    b    a
Enter 'q' to QUIT the game: d

Turno jugador uno

+----+----+----+----+----+----+----+----+
|    |  4 |  4 |  4 |  4 |  4 |  4 |    |
|  0 |----+----+----+----+----+----|  1 |
|    |  4 |  4 |  0 |  5 |  5 |  5 |    |
+----+----+----+----+----+----+----+----+
        f    e    d    c    b    a
Enter 'q' to QUIT the game: e
[4, 0, 1, 6, 6, 6, 1, 4, 4, 0, 5, 5, 5, 1]
###############################Se repite########################################################
La raíz es:[4, 0, 1, 6, 6, 6, 1, 4, 4, 0, 5, 5, 5, 1]
el valor es:0
Los valores son:[0, 0, 0, 0, 0]
Maximo valor es:0
[4, 0, 1, 6, 6, 6, 1, 4, 4, 0, 5, 5, 5, 1]
###############################Se repite########################################################
La raíz es:[